## Microtubule Dynamics ABM Julia notebook
This notebook is intended as a development and testing file , so i will change regulary.

nächstes meeting 05.10 10:45

### _TODO_ ::

    - Drei Abbildungen erstellen für klein 100, mittel300, groß1000 ,a 50 mal begrenzte randbedingungen ✓
    - seed funktion?!
    - Standartdabweichung (korridor) 
    - Rohdaten gut aufbewahren, sinnvolle beschriftungen  ✓
    - Periodischen Randbedingugen mit in microtubuli modell implementieren ✓

  - Wachstumsgeswindigkeit (erstmal ohne Depolymerisation) ~ N.Agenten/Gridsize & P-polymerisiert
  


In [66]:
using Agents        # for ABM
using CairoMakie    # static plotting 
using Distributions # 
using StatsBase: counts    
using DataFrames    # ?
using Statistics: mean
using InteractiveDynamics # plotting
using GLMakie       # plotting
using DrWatson: @dict #  model params, runtime, plotting
#include("Polymerize_new.jl")
include("Polymerize.jl")
using BenchmarkTools

In [22]:
@btime 1+1

  0.020 ns (0 allocations: 0 bytes)


2

In [55]:
#= The agent type representing 1 tubulin dimer
     @agent is a macro from the Agents.jl package that creates a struct with default keys
    :polym is a Int indicating which Microtubule the dimerpolymer is associated (0 = unpolymerized)
    :GDP::bool true = hydrolized (GDP -tubulin ) , false = GTP ("Cap") 
=#

@agent tubulin GridAgent{2} begin
    polym::Int64
    GDP::Bool
    # Plus::Int64
end

In [56]:
#= Main Function to initialize the model
Nstarts::Int number of seeding points
numagents::Int number of tubulin agents
griddims::Tuple(Int,2) Grid dimensions (please use a square one! and even number)
p_hyd:Float probability to hydrolize 
=#

function initialize(;Nstarts, numagents, periodic, griddims, p_hyd,  p_polym,
                        p_depolym_GTP,p_depolym_GDP)
   
    P_hyd = Binomial(1,p_hyd)                      # Prob to hydrolize from GTP to GDP:  GTPase rate
    P_polym = Binomial(1,p_polym)                  # Prob to Polymerize/bind to the microtuble 
    P_depolym_GTP = Binomial(1,1-p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    P_depolym_GDP = Binomial(1,1-p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher

    
 properties = @dict griddims numagents Nstarts p_depolym_GTP p_depolym_GDP p_polym p_hyd P_hyd P_polym P_depolym_GTP P_depolym_GDP
 
    properties[:tick] = 0

    space = GridSpace(griddims, periodic = periodic ) # initialize Gridspace
    
    model = ABM(tubulin, space;                     # create model using ?Agents.ABM
        scheduler = Schedulers.randomly,
        properties = properties )
    
    id = 0  #  populizing Grid Space with Agents  
    for _ in 1:Nstarts # Startingpoints:
        id += 1
        agent =  tubulin(id,griddims./2,id,true)
        if id == 1
            add_agent_pos!(agent,model)
        else
            add_agent_single!(agent,model) # add additional startingpoints to random positions 
        end
    end
    for _ in 1:numagents 
        id += 1
        agent =  tubulin(id,(1,1),0,false)
        add_agent_single!(agent,model) # add agents to random positions
    end
    return model
end

initialize (generic function with 1 method)

In [67]:
#= Agent Step function =#

function agent_step!(agent,model)
if agent.id > model.Nstarts
    if agent.polym == 0   # Logic for unpolymerized agents:    
            walk!(agent, rand, model, ifempty =true)
            polymerize(agent,model) 
        end
    if agent.polym > 0 # Logic for polymerized agents:    
        if agent.GDP == false   # if not hydrolized
            agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
            end
            depolymerize(agent,model)
        end
     end
end

agent_step! (generic function with 1 method)

In [68]:
#= Model Step function =#
function model_step!(model)
    model.tick += 1
    model.P_hyd = Binomial(1,model.p_hyd)                      # Prob to hydrolize from GTP to GDP: 
    model.P_polym = Binomial(1,model.p_polym)                  # Prob to Polymerize/bind to the microtuble 
    model.P_depolym_GTP = Binomial(1,1-model.p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    model.P_depolym_GDP = Binomial(1,1-model.p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher
end

model_step! (generic function with 1 method)

In [27]:
@btime step!(model,agent_step!,model_step! ,1)
# new

  4.856 ms (33723 allocations: 1.87 MiB)


In [61]:
@btime step!(model,agent_step!,model_step! ,10)
#old

  6.610 ms (39784 allocations: 1.94 MiB)


In [47]:
@btime step!(model,agent_step!,model_step! ,10)

#new2.0

  78.527 ms (490569 allocations: 27.39 MiB)


In [69]:
#= Model Initialisation =#

model = initialize(
    griddims =(100,100),
    periodic= true,
    Nstarts=4,
    numagents = 1000,
    p_polym=0.80,
    p_hyd= 0.02,
    p_depolym_GTP=0,
    p_depolym_GDP=0.03)

#step!(model,agent_step!,model_step! ,1, agents_first =true)

AgentBasedModel with 1004 agents of type tubulin
 space: GridSpace with size (100, 100), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: Dict{Symbol, Any}(:tick => 0, :griddims => (100, 100), :P_hyd => Binomial{Float64}(n=1, p=0.02), :p_depolym_GDP => 0.03, :Nstarts => 4, :P_depolym_GTP => Binomial{Float64}(n=1, p=1.0), :P_depolym_GDP => Binomial{Float64}(n=1, p=0.97), :p_polym => 0.8, :p_hyd => 0.02, :p_depolym_GTP => 0…)

In [62]:
@btime step!(model,agent_step!,model_step! ,1)

  650.059 μs (3964 allocations: 198.31 KiB)


In [36]:
@btime step!(model,agent_step!,model_step! ,1)
#old

  646.571 μs (3947 allocations: 197.62 KiB)


In [72]:
(2,1) in 2:3

false

In [70]:
# Interactive Model
using InteractiveDynamics
using GLMakie

params = Dict(
    :p_hyd => 0:0.01:1,
    :p_polym => 0:0.01:1,
    :p_depolym_GTP => 0:0.01:1,
    :p_depolym_GDP =>0:0.01:1
)
GLMakie.activate!()
# stats
# coloring
groupcolor(a) = a.polym >= 1 ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : :octagon
mdata=[mean_MT_size,sd_MT_size]
mlabels=["mean MT length", "sd"]
adata = [(:GDP,sum)]
alabels = ["Hydrolized"]

fig, adf, mdf = abm_data_exploration(
    model, agent_step!, model_step!, params;
    ac=groupcolor, am=groupmarker, adata,alabels,mdata,mlabels
)

(Scene (1600px, 800px):
  102 Plots:
    ├ Combined{Makie.poly, Tuple{Vector{Vector{Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Scatter{Tuple{Ve

In [ ]:
step!(model,agent_step!,model_step!,1)

In [ ]:
archie=random_agent(model)

In [ ]:
pa=periodic_agent_hood(archie,model)

In [ ]:
tuple(pa[1]...)

In [ ]:
# Neighborhood function for periodic Grids

function periodic_agent_hood(agent,model)
    agenthood=[[i,j] for i=-1:1 ,j=-1:1] 
    for i in 1:length(agenthood)
        agenthood[i] = agenthood[i] .+ agent.pos
        agenthood[i][agenthood[i].<1] .= model.griddims[1]
        agenthood[i]= mod1.(agenthood[i],model.griddims[1])
    end
    agenthood
end

In [7]:
# Data retrieval functions
# Function to calculate mean of microtubule length
function mean_MT_size(model)
    meanMTL=mean(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])
    # i remove the first index of counts ( polym==0 ) since this group doesnt represent a MT
end

# Function to calculate standart devation of microtubule length
function sd_MT_size(model)
    stdMTL=std(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])
end

sd_MT_size (generic function with 1 method)

In [8]:
### Data collection
#using Statistics: mean

adata = [:pos,:GDP,:polym,]                   # agent data
mdata=[:tick, mean_MT_size,sd_MT_size]
#data = run!(model,agent_step!,100;adata)

data, _ = run!(model,agent_step!,400;adata, mdata)



(402604×5 DataFrame
    Row │ step   id     pos       GDP    polym 
        │ Int64  Int64  Tuple…    Bool   Int64 
────────┼──────────────────────────────────────
      1 │     0      1  (50, 50)   true      1
      2 │     0      2  (97, 73)   true      2
      3 │     0      3  (34, 68)   true      3
      4 │     0      4  (23, 41)   true      4
      5 │     0      5  (9, 84)   false      0
      6 │     0      6  (95, 34)  false      0
      7 │     0      7  (55, 7)   false      0
      8 │     0      8  (35, 76)  false      0
      9 │     0      9  (78, 77)  false      0
     10 │     0     10  (95, 23)  false      0
     11 │     0     11  (4, 84)   false      0
   ⋮    │   ⋮      ⋮       ⋮        ⋮      ⋮
 402595 │   400    995  (13, 57)  false      0
 402596 │   400    996  (60, 20)  false      0
 402597 │   400    997  (26, 94)  false      0
 402598 │   400    998  (52, 82)  false      0
 402599 │   400    999  (54, 31)  false      0
 402600 │   400   1000  (69, 94)  false

In [ ]:
#= Scanning  parameters =#

parameters=Dict(
            :griddims => (80,80),
            :periodic => true,
            :Nstarts => 3,
            :numagents => [900,1000,1100],
            :p_hyd => 0.8,
            :p_polym=> [0.7,0.8,0.9] ,
            :p_depolym_GTP => 0,
            :p_depolym_GDP => [0,0.02,0.05,0.1]) 

In [ ]:
data,_ = paramscan(parameters, initialize;  mdata, agent_step!, model_step!, n=30)

In [ ]:
mod1(100,100)

In [ ]:
data[110:150,:]

In [ ]:
## Ensemble run RW
# Build ensemble of models

models = [initialize(; Nstarts=x,
        periodic= true,
        numagents = 1000,
        griddims=(x,x), 
        p_hyd=0,
        p_polym=0,
        p_depolym_GTP=0,
        p_depolym_GDP=0) for x in repeat([80, 100, 200], 30)];

adata = [:pos]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; adata mdata)
using CSV
CSV.write("data/ensemble1310.csv", adf)

In [ ]:
## Ensemble run Microtubuli simulation
models = [initialize(; Nstarts= x,
        periodic= true,
        numagents = 1000,
        griddims=(100,100), 
        p_polym=0.80,
        p_hyd= 0.02,
        p_depolym_GTP=0,
        p_depolym_GDP=0.03) for x in repeat([3,5,8,12], 20)];

mdata=[:tick, mean_MT_size,sd_MT_size]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; mdata)

In [ ]:
?paramscan


In [ ]:
## MSD for multiple ensembles

MSDS = DataFrame(Step= levels(adf.step))
for i in levels(adf.ensemble)
    name = Symbol("ens_$i")
    MSDS[!,name] = MSD(adf[(adf.ensemble .== i ),:])
end

CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", MSDS)

In [ ]:
CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", adf)